In [1]:
from seirsplus.models import *
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import json
import  pickle
from shapely.geometry import Polygon, Point, MultiPoint
from scipy.spatial import ConvexHull
from dateutil import parser

%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = (15, 8)

In [3]:
with open('region_sim3.p','rb') as fp:
    S5,S10,S20 = pickle.load(fp)

In [4]:
# Below section for creating graphs and monte carlo simul

def monte(N, initI,theta_E, theta_I ,mf, checkpoints, G, sim=2):
    beta = 0.185
    sigma = 1/5.2
    gamma = 1/15
    mu_I = 0.004
    p = 0.6
    Q = None
    beta_D = 0.155
    sigma_D = 1/5.2
    mu_D = 0.0004
    theta_E = theta_E
    theta_I = theta_I
    phi_E = 0.4
    phi_I = 0.8
    initN = N
    initI = initI
    initE = initI*4
    initD_E = 0.0
    initD_I = 0
    initR = 0
    initF = 0
    x_i = 0.001
    psi_E = 0.04
    psi_I = 1
    q = 0.3
    gamma_D = gamma

    # Creating dataframe
    time_index=pd.date_range(start='2020-04-01', periods=90)
    df_I=pd.DataFrame(index=time_index)
    df_DI=pd.DataFrame(index=time_index)
    df_R = pd.DataFrame(index=time_index)

    for i in range(1, sim+1):
        model = SEIRSNetworkModel(G = G,Q = G_Quarantine, beta = beta, sigma = sigma, gamma =gamma, mu_I = mu_I, beta_D = beta_D, sigma_D = sigma_D, 
                          gamma_D = gamma_D, mu_D = mu_D, theta_E = theta_E, theta_I = theta_I, phi_E = phi_E, phi_I = phi_I,
                          psi_E = psi_E, psi_I = psi_I, q = 0.3, initI = initI, initE = initE,initD_E = initD_E, 
                          initD_I = initD_I)



        model.run(T=120, checkpoints=checkpoints,verbose = True)
        time=pd.to_datetime(pd.Series(model.tseries),unit='d',origin='2020-04-01')

        df_tempI=pd.DataFrame({'t':time, i:model.numI})
        df_tempI.set_index('t',inplace=True)
        df_tempI['t']=df_tempI.index.date
        df_tempI.reset_index(drop=True,inplace=True)
        df_tempI.set_index('t',inplace=True)
        df_tempI.reset_index(inplace=True)
        df_tempI = pd.pivot_table(df_tempI, index='t', aggfunc='mean')

        df_I=df_I.merge(df_tempI, how='outer', right_index=True, left_index=True).fillna(np.NaN)


        df_tempDI=pd.DataFrame({'t':time, i:model.numD_I})
        df_tempDI.set_index('t',inplace=True)
        df_tempDI['t']=df_tempDI.index.date
        df_tempDI.reset_index(drop=True,inplace=True)
        df_tempDI.set_index('t',inplace=True)
        df_tempDI.reset_index(inplace=True)
        df_tempDI= pd.pivot_table(df_tempDI, index='t', aggfunc='mean')
        

        df_DI=df_DI.merge(df_tempDI, how='outer', right_index=True, left_index=True).fillna(np.NaN)
        
        df_tempR=pd.DataFrame({'t':time, i:model.numR})
        df_tempR.set_index('t',inplace=True)
        df_tempR['t']=df_tempR.index.date
        df_tempR.reset_index(drop=True,inplace=True)
        df_tempR.set_index('t',inplace=True)
        df_tempR.reset_index(inplace=True)
        df_tempR= pd.pivot_table(df_tempR, index='t', aggfunc='mean')
        

        df_R=df_R.merge(df_tempR, how='outer', right_index=True, left_index=True).fillna(np.NaN)
        
        
        df_I =df_I.interpolate(method='linear', limit_direction='forward', axis=0)
        df_DI =df_DI.interpolate(method='linear', limit_direction='forward', axis=0)
        df_R = df_R.interpolate(method='linear',limit_direction='forward',axis = 0)
        
        # Changes for looking at actual numbers
        df_Iavg=pd.DataFrame.mean(df_I, axis=1)
        df_Iavg = df_Iavg/N
        df_DIavg=pd.DataFrame.mean(df_DI, axis=1)
        df_DIstd=pd.DataFrame.std(df_DI.cumsum(axis=1), axis=1)/N
        df_DIavg = df_DIavg/N
        df_Ravg = pd.DataFrame.mean(df_R, axis = 1)
        df_Ravg = df_Ravg/N
        
        
    return {'df_DI':df_DIavg, 'df_I': df_Iavg,'df_R': df_Ravg, 'df_std':df_DIstd}

In [5]:
def gen_graphs(numNodes, coeff):
    G_normal = nx.erdos_renyi_graph(n=numNodes, p = coeff*1.1*np.log(5000)/numNodes, seed=42, directed=False)
    G_relaxed = nx.erdos_renyi_graph(n=numNodes, p = coeff*0.301*np.log(5000)/numNodes , seed=42, directed=False)
    G_Quarantine = nx.erdos_renyi_graph(n=numNodes, p = coeff*0.006*np.log(5000)/numNodes , seed=42, directed=False)
    
    return G_normal, G_relaxed, G_Quarantine

In [6]:
I = 2
Et = [0.000, 0.0000,0.0002,0.0002]
It = [0.00004, 0.00054, 0.001, 0.004]
sim = 1000

In [ ]:
# With current testing and social distancing throughout
numNodes = S5['total_pop']
G_normal, G_relaxed, G_Quarantine = gen_graphs(numNodes, 1)

final = []
initI = I
theta_E = Et[0]
theta_I = It[0]
mf = 1
checkpoints  =  {'t': [15,25,50],
                   'theta_E': Et[1:],
                   'theta_I': It[1:],
                 'G': [G_relaxed,G_Quarantine,G_Quarantine]
                      }

temp = monte(numNodes,initI=initI,theta_E = theta_E, theta_I = theta_I ,mf=mf, checkpoints=checkpoints, G = G_normal, sim=sim);

t = 1.00
	 S   = 5080.0
	 E   = 7.0
	 I   = 3.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 0.0
	 F   = 0.0
t = 10.05
	 S   = 5067.0
	 E   = 7.0
	 I   = 11.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 5.0
	 F   = 0.0
[Checkpoint: Updating parameters]
t = 20.03
	 S   = 5055.0
	 E   = 3.0
	 I   = 16.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 15.0
	 F   = 1.0
[Checkpoint: Updating parameters]
t = 30.44
	 S   = 5046.0
	 E   = 1.0
	 I   = 13.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 29.0
	 F   = 1.0
[Checkpoint: Updating parameters]
t = 0.41
	 S   = 5080.0
	 E   = 7.0
	 I   = 3.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 0.0
	 F   = 0.0
t = 10.22
	 S   = 5069.0
	 E   = 6.0
	 I   = 9.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 5.0
	 F   = 1.0
[Checkpoint: Updating parameters]
t = 20.62
	 S   = 5065.0
	 E   = 1.0
	 I   = 12.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 11.0
	 F   = 1.0
[Checkpoint: Updating parameters]
t = 30.32
	 S   = 5061.0
	 E   = 2.0
	 I   = 12.0
	 D_E = 0.0
	 D_I = 0.0
	 R   = 12.0
	 F   = 3.0
t = 40.31
	 S   = 5061.0
	 E   = 0.0
	 I   = 10

In [ ]:
cumu = temp['df_DI'].cumsum(axis=0)
plt.plot(cumu)
plt.plot(cumu - 1.96*temp['df_std']/np.sqrt(sim), 'r--')
plt.plot(cumu + 1.96*temp['df_std']/np.sqrt(sim), 'r--')
plt.plot(
    list(cumu.keys())[:len(S10['timeseries'][15:])],
    [float(j[1])/numNodes for j in S5['timeseries'][15:]], 'g')

In [ ]:
coeff = (S10['total_pop']*np.log(S10['total_pop'])*S5['area']*S5['F(S)']**2)/(S5['total_pop']*np.log(S5['total_pop'])*S10['area']*S10['F(S)']**2)

In [ ]:
print(coeff)

In [ ]:
# With current testing and social distancing throughout
numNodes = S10['total_pop']
G_normal, G_relaxed, G_Quarantine = gen_graphs(numNodes, coeff)

final = []
initI = int(I*coeff)
theta_E = Et[0]
theta_I = It[0]
mf = 1
checkpoints  =  {'t': [15,25,50],
                   'theta_E': Et[1:],
                   'theta_I': It[1:],
                 'G': [G_relaxed,G_Quarantine,G_relaxed]
                      }

temp = monte(numNodes,initI=initI,theta_E = theta_E, theta_I = theta_I ,mf=mf, checkpoints=checkpoints, G = G_normal, sim=sim);

In [ ]:
cumu = temp['df_DI'].cumsum(axis=0)
plt.plot(cumu)
plt.plot(cumu - 1.96*temp['df_std']/np.sqrt(sim), 'r--')
plt.plot(cumu + 1.96*temp['df_std']/np.sqrt(sim), 'r--')
plt.plot(
    list(cumu.keys())[:len(S10['timeseries'][15:])],
    [float(j[1])/numNodes for j in S10['timeseries'][15:]], 'g')

In [ ]:
coeff = (S20['total_pop']*np.log(S20['total_pop'])*S5['area']*S5['F(S)']**2)/(S5['total_pop']*np.log(S5['total_pop'])*S20['area']*S20['F(S)']**2)

In [ ]:
print(coeff)

In [ ]:
# With current testing and social distancing throughout
numNodes = S20['total_pop']
G_normal, G_relaxed, G_Quarantine = gen_graphs(numNodes, coeff)


final = []
initI = int(I*coeff)
theta_E = Et[0]
theta_I = It[0]
mf = 1
checkpoints  =  {'t': [15,25,50],
                   'theta_E': Et[1:],
                   'theta_I': It[1:],
                 'G': [G_relaxed,G_Quarantine,G_Quarantine]
                      }

temp = monte(numNodes,initI=initI,theta_E = theta_E, theta_I = theta_I ,mf=mf, checkpoints=checkpoints, G = G_normal, sim=sim);

In [ ]:
cumu = temp['df_DI'].cumsum(axis=0)
plt.plot(cumu)
plt.plot(cumu - 1.96*temp['df_std']/np.sqrt(sim), 'r--')
plt.plot(cumu + 1.96*temp['df_std']/np.sqrt(sim), 'r--')
plt.plot(
    list(cumu.keys())[:len(S10['timeseries'][15:])],
    [float(j[1])/numNodes for j in S20['timeseries'][15:]], 'g')